<a href="https://colab.research.google.com/github/Rajdeep7102/Rajdeep7102/blob/main/face_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import zipfile
# Unzip our data
zip_ref = zipfile.ZipFile("/content/Face-Images.zip","r")
zip_ref.extractall()
zip_ref.close()

In [4]:
import os

# Walk through Face-Images directory and list number of files
for dirpath, dirnames, filenames in os.walk("Face-Images"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

In [16]:
TrainingImagePath = 'Face Images//Final Training Images'

In [ ]:
import keras
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True)

# Defining pre-processing transformations on raw images of testing data
# No transformations are done on the testing images
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator()




# Create an augmented data generator instance
train_datagen_augmented = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.,
                                             rotation_range=0.2,
                                             width_shift_range=0.2,
                                             height_shift_range=0.2,
                                             zoom_range=0.2,
                                             horizontal_flip=True)

# Generating the Training Data
training_set = train_datagen_augmented.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

# Generating the Testing Data
test_set = test_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

# Printing class labels for each face
test_set.class_indices


In [34]:
training_set = train_datagen_augmented.flow_from_directory(TrainingImagePath,target_size=(64,64),batch_size=32,class_mode='categorical')

test_set = test_datagen.flow_from_directory(TrainingImagePath,target_size=(64,64),batch_size=32,
                                            class_mode='categorical')

test_set.class_indices

Found 244 images belonging to 16 classes.
Found 244 images belonging to 16 classes.


{'face1': 0,
 'face10': 1,
 'face11': 2,
 'face12': 3,
 'face13': 4,
 'face14': 5,
 'face15': 6,
 'face16': 7,
 'face2': 8,
 'face3': 9,
 'face4': 10,
 'face5': 11,
 'face6': 12,
 'face7': 13,
 'face8': 14,
 'face9': 15}

In [35]:
TrainClasses = training_set.class_indices

ResultMap={}
for faceValue, faceName in zip(TrainClasses.values(),TrainClasses.keys()):
  ResultMap[faceValue]=faceName

import pickle
with open("ResultsMap.pkl",'wb') as fileWriteStream:
  pickle.dump(ResultMap,fileWriteStream)

print("Mapping of face and its ID",ResultMap)

OutputNeurons=len(ResultMap)
print('\n The Number of output neurons: ', OutputNeurons)

Mapping of face and its ID {0: 'face1', 1: 'face10', 2: 'face11', 3: 'face12', 4: 'face13', 5: 'face14', 6: 'face15', 7: 'face16', 8: 'face2', 9: 'face3', 10: 'face4', 11: 'face5', 12: 'face6', 13: 'face7', 14: 'face8', 15: 'face9'}

 The Number of output neurons:  16


In [42]:
'''######################## Create CNN deep learning model ########################'''
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense

'''Initializing the Convolutional Neural Network'''
classifier= Sequential()

''' STEP--1 Convolution
# Adding the first layer of CNN
# we are using the format (64,64,3) because we are using TensorFlow backend
# It means 3 matrix of size (64X64) pixels representing Red, Green and Blue components of pixels
'''
classifier.add(Convolution2D(32, kernel_size=(5, 5), strides=(1, 1), input_shape=(64,64,3), activation='relu'))

'''# STEP--2 MAX Pooling'''
classifier.add(MaxPool2D(pool_size=(2,2)))

'''############## ADDITIONAL LAYER of CONVOLUTION for better accuracy #################'''
classifier.add(Convolution2D(64, kernel_size=(5, 5), strides=(1, 1), activation='relu'))

classifier.add(MaxPool2D(pool_size=(2,2)))

'''# STEP--3 FLattening'''
classifier.add(Flatten())

'''# STEP--4 Fully Connected Neural Network'''
classifier.add(Dense(64, activation='relu'))

classifier.add(Dense(OutputNeurons, activation='softmax'))

'''# Compiling the CNN'''
#classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
classifier.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=["accuracy"])

###########################################################
import time
# Measuring the time taken by the model to train
StartTime=time.time()

# Starting the model training
classifier.fit_generator(
                    training_set,
                    steps_per_epoch=8,
                    epochs=100,
                    validation_data=test_set,
                    validation_steps=10)

EndTime=time.time()
print("###### Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes ######')

Epoch 1/100


<ipython-input-42-b0c303e2d1c4>:44: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(


8/8 [==============================] - ETA: 0s - loss: 2.8110 - accuracy: 0.0533

8/8 [==============================] - 5s 457ms/step - loss: 2.8110 - accuracy: 0.0533 - val_loss: 23.6183 - val_accuracy: 0.0656
Epoch 2/100
8/8 [==============================] - 4s 443ms/step - loss: 2.7722 - accuracy: 0.0697
Epoch 3/100
8/8 [==============================] - 3s 299ms/step - loss: 2.7535 - accuracy: 0.0820
Epoch 4/100
8/8 [==============================] - 3s 291ms/step - loss: 2.7197 - accuracy: 0.0861
Epoch 5/100
8/8 [==============================] - 3s 297ms/step - loss: 2.6964 - accuracy: 0.1230
Epoch 6/100
8/8 [==============================] - 3s 299ms/step - loss: 2.6592 - accuracy: 0.1680
Epoch 7/100
8/8 [==============================] - 3s 298ms/step - loss: 2.5462 - accuracy: 0.1475
Epoch 8/100
8/8 [==============================] - 3s 294ms/step - loss: 2.4423 - accuracy: 0.2459
Epoch 9/100
8/8 [==============================] - 3s 349ms/step - loss: 2.2752 - accuracy: 0.2418
Epoch 10/100
8/8 [==============================] - 4s 396ms/step - loss: 2.33

In [49]:
import numpy as np
from keras.preprocessing import image

ImagePath = '/content/Face Images/Final Testing Images/face14/4face4.jpg'
test_image = image.load_img(ImagePath,target_size=(64,64))
test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)
result = classifier.predict(test_image,verbose=0)
print('Prediction is:',ResultMap[np.argmax(result)])


Prediction is: face14
